In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [2]:
# data loading config
batch_size = 64
img_height = 100
img_width = 100
labels = 'inferred'
label_mode = 'categorical'  # sparse one hot encoding
color_mode = 'rgb'
shuffle = True
seed = 69
val_split = 0.2
AUTOTUNE = tf.data.AUTOTUNE

print('Training data:')
train = tf.keras.utils.image_dataset_from_directory('train', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
train_class_names = train.class_names

train = train.cache().prefetch(buffer_size=AUTOTUNE)

Training data:
Found 3081 files belonging to 34 classes.


In [3]:
# model layers
xIn = Input((img_height, img_width, 3))
# x = tf.keras.layers.RandomFlip()(xIn)
effnet = tf.keras.applications.efficientnet_v2.EfficientNetV2M(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3),)
x = effnet(xIn)
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='swish')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='swish')(x)
x = Dropout(0.3)(x)
xOut = Dense(len(train_class_names), dtype=tf.float32)(x)


model = Model(inputs=xIn, outputs=xOut)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=['accuracy'])

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=8, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=5, verbose=1)
]

model.summary()
model.fit(train, epochs=100, batch_size=64, callbacks=callbacks)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 efficientnetv2-m (Functiona  (None, 4, 4, 1280)       53150388  
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 20480)             0         
                                                                 
 dropout (Dropout)           (None, 20480)             0         
                                                                 
 dense (Dense)               (None, 256)               5243136   
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                             

In [4]:
model.save('comnist_full')

INFO:tensorflow:Assets written to: comnist_full\assets


In [3]:
model = load_model('comnist')
model2 = load_model('comnist_aug')
model3 = load_model('comnist_1')
model4 = load_model('comnist_full')

In [4]:
test = tf.keras.utils.image_dataset_from_directory('test', labels=None, label_mode=None,
color_mode=color_mode, shuffle=False, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
filenames = test.file_paths
filenames = [file.split('\\')[-1].split('.jpg')[0] for file in filenames]
test = test.cache().prefetch(buffer_size=AUTOTUNE)
preds = model.predict(test)
preds = tf.nn.softmax(preds)
preds2 = model2.predict(test)
preds2 = tf.nn.softmax(preds2)
preds3 = model3.predict(test)
preds3 = tf.nn.softmax(preds3)
preds4 = model4.predict(test)
preds4 = tf.nn.softmax(preds4)

Found 12399 files belonging to 1 classes.


In [67]:
preds_comb = preds * 0.2 +  preds2 * 0.15 + preds3 * 0.3 + preds4 * 0.35  # 9748
preds_comb = tf.argmax(preds_comb, axis=-1)

In [68]:
preds_dict = dict()
for i in range(len(preds)):
    preds_dict.update({filenames[i]: train_class_names[int(preds_comb[i])]})

pred_df = pd.read_csv('test.csv')
pred_df_unordered = pd.DataFrame.from_dict(preds_dict, columns=['label'], orient='index')
pred_df_unordered.reset_index(inplace=True)
pred_df_unordered.columns = ['id', 'label']

pred_df_aligned = pred_df.merge(pred_df_unordered, on='id', how='left')
pred_df_aligned.drop(['label_x'], axis=1, inplace=True)
pred_df_aligned.columns = ['id', 'label']
pred_df_aligned.to_csv('submission.csv', index=False)

In [69]:
# Connect to graders
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, 'C:/Users/alien/Documents/PyCharm Projects/Cyberthon 2021/pyctfsglib.py')
import pyctfsglib as ctfsg
import random

USER_TOKEN = "WrlLCkymxwtgFwRHZsdmKfSwcdqIpnqoXEtRkciVRZJfBJUgcEJoxVZjNTQRdqkR" # You need to fill this up
GRADER_URL = random.choice([
  "http://chals.cyberthon22t.ctf.sg:50201/",
  "http://chals.cyberthon22t.ctf.sg:50202/"
])

grader = ctfsg.DSGraderClient(GRADER_URL, USER_TOKEN)
grader.submitFile('submission.csv')

DSGraderClient: Successfully Connected!
[SERVER] MOTD: CHECK your USER_TOKEN and GRADER_URL HTTP address! I'm CoMNIST @4d907f5f7099
ProofOfWork Challenge =>  ('CTFSGRB118c314edff3f312ce7f933fec6ef25d', 22)
ProofOfWork Answer Found! =>  4855291


'{"challenge":{"name":"CoMNIST"},"id":"cl232xxdzd9jw0827u6jbxpsq","status":"PARTIALLY_CORRECT","multiplier":0.9748,"submittedBy":{"username":"hci-69"},"createdAt":"2022-04-17T09:22:31Z"}'